In [38]:
import sys
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
from matplotlib import pylab as plt

import sklearn
sklearn.set_config(enable_metadata_routing=True)

from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.subsample import BlockBootstrap
from mapie.regression import MapieTimeSeriesRegressor, MapieRegressor

import torch
import json
import pickle

from sklearn.metrics import mean_squared_error

In [4]:
sys.path.append(r'C:\Users\obhlivoj\DP\System-Imbalance-Forecasting\models\xgboost')

from config import get_config
from train import get_ds, get_model

path = r'C:\Users\obhlivoj\DP\System-Imbalance-Forecasting\models\xgboost'

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = get_config()

train, val, test = get_ds(cfg)
x_train, y_train, hist_train = train
x_val, y_val, hist_val = val
x_test, y_test, hist_test = test

# read json_info
with open(f'{path}/final_params.json', 'r') as file:
    best_params = json.load(file)

for param, value in best_params['best_params'][0].items():
    cfg[param] = value

cfg['early_stopping_rounds'] = None
model = get_model(cfg, device)
# Load the pretrained weights
# model.load_model(f"{path}/weights_folder/weightsforward_1/xgb_model_1.json")

# config updates
cfg['run'] = f"mapie_test"
cfg['model_folder'] += cfg['run']

In [30]:
cv_mapiets = BlockBootstrap(
    n_resamplings=4, n_blocks=10, overlapping=False, random_state=69
)

alpha = 0.05

In [31]:
x_calib = np.concatenate((x_train, x_val), axis = 0)
y_calib = np.concatenate((y_train, y_val), axis = 0)

cfg['n_estimators'] = 300

mapie_ts = MapieTimeSeriesRegressor(
    model, method="enbpi", cv=cv_mapiets, agg_function="mean", n_jobs=-1
)
mapie_ts.fit(x_calib, y_calib.squeeze(-1))

MapieTimeSeriesRegressor(cv=BlockBootstrap(length=None, n_blocks=10, n_resamplings=4, overlapping=False,
        random_state=69),
                         estimator=XGBRegressor(base_score=None, booster=None,
                                                callbacks=None,
                                                colsample_bylevel=None,
                                                colsample_bynode=None,
                                                colsample_bytree=None,
                                                device='cuda',
                                                early_stopping_rounds=None,
                                                enable_categorical=False,
                                                eval_metric=['mae', 'rmse'],
                                                featu...
                                                grow_policy=None,
                                                importance_type=None,
                                                interaction_constraints=None,
                                                learning_rate=0.06,
                                                max_bin=None,
                                                max_cat_threshold=None,
                                                max_cat_to_onehot=None,
                                                max_delta_step=None,
                                                max_depth=4, max_leaves=None,
                                                min_child_weight=None,
                                                missing=nan,
                                                monotone_constraints=None,
                                                multi_strategy=None,
                                                n_estimators=750, n_jobs=None,
                                                num_parallel_tree=None,
                                                random_state=None, ...),
                         n_jobs=-1)

In [33]:
y_pred, y_pis = mapie_ts.predict(x_test, alpha=alpha)
coverage = regression_coverage_score(y_test, y_pis[:, 0, 0], y_pis[:, 1, 0])
width = regression_mean_width_score(y_pis[:, 0, 0], y_pis[:, 1, 0])

In [41]:
mean_squared_error(mapie_ts.predict(x_test), y_test, squared=False)

109.324455

In [40]:
width

419.9063148498535

In [45]:
xgb_res = {
    "y_pred": y_pred,
    "y_pis": y_pis,
    "coverage": coverage,
    "width": width,
}

with open('./results/xgb_dict.pkl', 'wb') as f:
    pickle.dump(xgb_res, f)